In [1]:
from ftx_client import FtxClient
import pandas as pd
import datetime

In [2]:
ftx = FtxClient(api_key="xxxxx",
                api_secret="xxxxx")

In [3]:
account_info = ftx.get_balances()
account_info = pd.DataFrame(account_info)
account_info.set_index("coin", inplace=True)

In [4]:
account_info

,total,free,availableWithoutBorrow,usdValue,spotBorrow
coin,,,,,
ETH,1.167968,0.025415,5.400000e-07,4.680080e+03,0.0
FTT,3.000000,3.000000,3.000000e+00,1.227120e+02,0.0
USD,0.000000,107.196766,0.000000e+00,0.000000e+00,0.0
BTC,0.025123,0.002134,2.797000e-05,1.232611e+03,0.0
TRX,0.000002,1076.341437,2.000000e-06,1.792686e-07,0.0
USDT,0.000000,104.457821,0.000000e+00,5.659828e-09,0.0


In [5]:
account_info

,total,free,availableWithoutBorrow,usdValue,spotBorrow
coin,,,,,
ETH,1.167968,0.025415,5.400000e-07,4.680080e+03,0.0
FTT,3.000000,3.000000,3.000000e+00,1.227120e+02,0.0
USD,0.000000,107.196766,0.000000e+00,0.000000e+00,0.0
BTC,0.025123,0.002134,2.797000e-05,1.232611e+03,0.0
TRX,0.000002,1076.341437,2.000000e-06,1.792686e-07,0.0
USDT,0.000000,104.457821,0.000000e+00,5.659828e-09,0.0


In [6]:
print(account_info[account_info.index != "BTC"])

         total         free  availableWithoutBorrow      usdValue  spotBorrow
coin                                                                         
ETH   1.167968     0.025415            5.400000e-07  4.680080e+03         0.0
FTT   3.000000     3.000000            3.000000e+00  1.227120e+02         0.0
USD   0.000000   107.196766            0.000000e+00  0.000000e+00         0.0
TRX   0.000002  1076.341437            2.000000e-06  1.792686e-07         0.0
USDT  0.000000   104.457821            0.000000e+00  5.659828e-09         0.0


In [7]:
coin_name_list = account_info.index.tolist()

In [8]:
df  = pd.DataFrame(account_info[(account_info.index != "USD") & (account_info.index != "USDT")])
coin_name_list = df.index.tolist()

In [9]:
end_time = datetime.datetime.now().timestamp()
period = 7
markets = ftx.get_market_price_history(market='BTC/USDT', 
                resolution=86400, 
                start_time=end_time-(86400*int(period)), 
                end_time=end_time)

In [10]:
markets = pd.DataFrame(markets)
markets

,startTime,time,open,high,low,close,volume
0,2021-12-07T00:00:00+00:00,1.638835e+12,50450.0,51934.0,50022.0,50593.0,1.531147e+08
1,2021-12-08T00:00:00+00:00,1.638922e+12,50593.0,51210.0,48616.0,50476.0,1.656571e+08
2,2021-12-09T00:00:00+00:00,1.639008e+12,50476.0,50812.0,47324.0,47546.0,2.970996e+08
3,2021-12-10T00:00:00+00:00,1.639094e+12,47546.0,50092.0,46888.0,47153.0,2.369254e+08
4,2021-12-11T00:00:00+00:00,1.639181e+12,47153.0,49500.0,46762.0,49388.0,1.494919e+08
5,2021-12-12T00:00:00+00:00,1.639267e+12,49388.0,50784.0,48633.0,50065.0,1.299888e+08
6,2021-12-13T00:00:00+00:00,1.639354e+12,50065.0,50190.0,48442.0,49045.0,2.782313e+07


In [11]:
markets["netValue"] = markets["close"] * account_info.loc['BTC','total'] 

In [12]:
markets

,startTime,time,open,high,low,close,volume,netValue
0,2021-12-07T00:00:00+00:00,1.638835e+12,50450.0,51934.0,50022.0,50593.0,1.531147e+08,1271.046421
1,2021-12-08T00:00:00+00:00,1.638922e+12,50593.0,51210.0,48616.0,50476.0,1.656571e+08,1268.107034
2,2021-12-09T00:00:00+00:00,1.639008e+12,50476.0,50812.0,47324.0,47546.0,2.970996e+08,1194.496732
3,2021-12-10T00:00:00+00:00,1.639094e+12,47546.0,50092.0,46888.0,47153.0,2.369254e+08,1184.623404
4,2021-12-11T00:00:00+00:00,1.639181e+12,47153.0,49500.0,46762.0,49388.0,1.494919e+08,1240.773242
5,2021-12-12T00:00:00+00:00,1.639267e+12,49388.0,50784.0,48633.0,50065.0,1.299888e+08,1257.781493
6,2021-12-13T00:00:00+00:00,1.639354e+12,50065.0,50190.0,48442.0,49045.0,2.782313e+07,1232.156064


In [13]:
#function create history price df
def create_history_price_df():
    end_time = datetime.datetime.now().timestamp()
    period = 7
    account_info = pd.DataFrame(ftx.get_balances())
    close_price_df = pd.DataFrame()
    account_info.set_index("coin", inplace=True)  
    mask_usd_df = pd.DataFrame(account_info[(account_info.index != "USD") & (account_info.index != "USDT")])
    markets = mask_usd_df.index.tolist()
    for market in markets:
        json_data_from_ftx = ftx.get_market_price_history(market='{}/USDT'.format(market), 
                    resolution=86400, 
                    start_time=end_time-(86400*int(period)), 
                    end_time=end_time)
        df_data = pd.DataFrame(json_data_from_ftx)
        close_price_df[market+'_value'] = df_data["close"] * account_info.loc[market, 'total']
    close_price_df['USDT'] = account_info.loc['USDT', 'total']
    close_price_df['sum'] = close_price_df.sum(axis=1)
    return(close_price_df)

In [15]:
df = create_history_price_df()

In [16]:
df

,ETH_value,FTT_value,BTC_value,TRX_value,USDT,sum
0,5029.852211,128.640,1271.046421,1.799700e-07,0.0,6429.538632
1,5183.089552,132.051,1268.107034,1.846100e-07,0.0,6583.247586
2,4792.988394,120.531,1194.496732,1.772800e-07,0.0,6108.016126
3,4553.555048,115.926,1184.623404,1.753500e-07,0.0,5854.104453
4,4775.819271,121.149,1240.773242,1.831250e-07,0.0,6137.741514
5,4823.822737,123.792,1257.781493,1.820550e-07,0.0,6205.396230
6,4676.191640,122.619,1232.181187,1.791250e-07,0.0,6030.991827
